In [8]:
from __future__ import  division
import pandas as pd
import numpy as np

from math import sin, cos, sqrt, atan2, radians, degrees, fabs
import string as str
import re
from random import randint

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection 
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import GaussianMixture


import xgboost as xgb
from nltk.stem import PorterStemmer

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print 'Files imported'

#creating test_train and y_train
y = train_df['is_duplicate'].values

y_train = np.array(y)


print 'Creating features' 

x_tr = np.loadtxt("x_train.gz", delimiter=",")
x_ts = np.loadtxt("x_test.gz", delimiter=",")


Files imported
Creating features


In [9]:
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

x_tr[~np.isfinite(x_tr)] = 0
# x_test[~np.isfinite(x_test)] = 0

clf = RandomForestClassifier(n_estimators = 100, n_jobs =-1)
clf = clf.fit(x_tr, y_train)
clf.feature_importances_  

model = SelectFromModel(clf,threshold = 'median', prefit=True)



In [10]:
X_new = model.transform(x_tr)

X_new.shape               


(404290, 106)

In [ ]:
RS = 2016
ROUNDS = 400

print("Started")
np.random.seed(RS)
input_folder = ''

# x_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RS)

import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.11
params['max_depth'] = 5
params['seed'] = RS
params['gamma'] = 0.5
params['subsample'] = 0.75
params['colsample_bytree'] = 0.75
params['min_child_weight'] = 10
params['reg_alpha'] = 2

from sklearn import model_selection
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

x_train = X_new
# y_train = np.array(y)


train_stacker=[ [0.0 for s in range(1)]  for k in range (0,(x_train.shape[0])) ]

cv_scores = []
oof_preds = []
a = [0 for x in range(2345796)]
# StratifiedKFold
# kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=RS)
# for dev_index, val_index in kf.split(range(x_train.shape[0]),y_train):
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(x_train.shape[0])):
        dev_X, val_X = x_train[dev_index,:], x_train[val_index,:]
        dev_y, val_y = y_train[dev_index], y_train[val_index]
        print dev_X.shape
        print val_X.shape
        
        pos_train = dev_X[dev_y == 1]
        neg_train = dev_X[dev_y == 0]

        print("Oversampling started for proportion: {}".format(len(pos_train) / (len(pos_train) + len(neg_train))))
        p = 0.165
        scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
        while scale > 1:
            neg_train = np.concatenate((neg_train, neg_train))
            scale -=1
        neg_train = np.concatenate((neg_train, neg_train[:int(scale * len(neg_train))]))
        print("Oversampling done, new proportion: {}".format(len(pos_train) / (len(pos_train) + len(neg_train))))

        dev_X = np.concatenate((pos_train, neg_train))
        dev_y = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
        del pos_train, neg_train  

        pos_train = val_X[val_y == 1]
        neg_train = val_X[val_y == 0]

        print("Oversampling started for proportion: {}".format(len(pos_train) / (len(pos_train) + len(neg_train))))
        p = 0.165
        scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
        while scale > 1:
            neg_train = np.concatenate((neg_train, neg_train))
            scale -=1
        neg_train = np.concatenate((neg_train, neg_train[:int(scale * len(neg_train))]))
        print("Oversampling done, new proportion: {}".format(len(pos_train) / (len(pos_train) + len(neg_train))))

        val_X = np.concatenate((pos_train, neg_train))
        val_y = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
        del pos_train, neg_train  

        print dev_X.shape
        print val_X.shape

        d_train = xgb.DMatrix(dev_X, label=dev_y)
        d_valid = xgb.DMatrix(val_X, label=val_y)

        watchlist = [(d_train, 'train'), (d_valid, 'valid')]

        bst = xgb.train(params, d_train, 15000, watchlist, early_stopping_rounds=50, verbose_eval=100)

        preds = bst.predict(d_valid)

        cv_scores.append(log_loss(val_y, preds))
        
#         d_test = xgb.DMatrix(x_test)
#         preds_tr = bst.predict(d_test)

#         a = np.column_stack((a,preds_tr))
#         cv_scores.append(log_loss(val_y, preds*0.99))
#         cv_scores.append(log_loss(val_y, preds*0.98))
#         cv_scores.append(log_loss(val_y, preds*0.95))
#         cv_scores.append(log_loss(val_y, preds*0.90))

        print(cv_scores)
        break

#         no=0
#         for real_index in val_index:
# #             for d in range (0,1):
#             train_stacker[real_index]= preds
#             no+=1

Started
(323432, 106)
(80858, 106)
Oversampling started for proportion: 0.369026565089
Oversampling done, new proportion: 0.191259632688
Oversampling started for proportion: 0.369883004774
Oversampling done, new proportion: 0.191181170815
(624047, 106)
(156438, 106)
[0]	train-logloss:0.618231	valid-logloss:0.61811
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[100]	train-logloss:0.183504	valid-logloss:0.188552
[700]	train-logloss:0.143527	valid-logloss:0.176013
[800]	train-logloss:0.139219	valid-logloss:0.175555
[900]	train-logloss:0.135095	valid-logloss:0.175499
[1000]	train-logloss:0.131197	valid-logloss:0.175415
[1100]	train-logloss:0.12742	valid-logloss:0.175257
[1200]	train-logloss:0.123785	valid-logloss:0.175178
Stopping. Best iteration:
[1226]	train-logloss:0.122829	valid-logloss:0.175126

[0.17517136348937903]
